# Dinâmica simbólica automáticaNotebook reduzido para duas etapas: clonar o repositório e calcular as equações demovimento com geração automática das variáveis a partir do vetor de rotações e dasmatrizes de excentricidade/inércia, seguindo o mesmo método usado no MATLAB.

## 1. Clonar o repositórioAltere `repo_url` caso queira apontar para outro fork. Se o notebook já estiver dentrodo repositório, nada é clonado.

In [ ]:
# URL do repositório Gitrepo_url = "https://github.com/danisilva1110-spec/PFPython.git"from pathlib import Pathimport sysimport subprocesscurrent_repo = Path.cwd()local_package = current_repo / "robot_dynamics"if local_package.exists():    repo_path = current_repoelse:    repo_dir = repo_url.rstrip("/").split("/")[-1]    if repo_dir.endswith(".git"):        repo_dir = repo_dir[:-4]    repo_path = current_repo / repo_dir    if not repo_path.exists():        subprocess.run(["git", "clone", repo_url], check=True)    sys.path.insert(0, str(repo_path))

## 2. Gerar variáveis e tabelas automaticamenteDefina o vetor de rotações/translações (`axis_order`) e as matrizes de excentricidade einércia. Entradas `0` travam a variável, `1` criam um símbolo e qualquer outro valorfixa o termo. A máscara `active_mask` segue a mesma lógica 0/1 para ativar ou desativarum grau de liberdade inteiro.

In [ ]:
from robot_dynamics import auto_parameters_from_order# Ordem do UVMS (ROV 6DOF + braço 6DOF)axis_order = ["Dx", "Dy", "Dz", "z", "y", "x", "z", "y", "y", "z", "y", "z"]# Máscara 0/1 para ligar/desligar juntas (opcional)active_mask = [1] * len(axis_order)# Excentricidades (0 = sem deslocamento; 1 = símbolo automático)excentricity_template = [    (0, 0, 0),  # Dx    (0, 0, 0),  # Dy    (0, 0, 0),  # Dz    (0, 0, 0),  # Rz    (0, 0, 0),  # Ry    (0, 0, 0),  # Rx    (0, 0, 1),  # p1    (1, 0, 0),  # p2    (1, 0, 0),  # p3    (1, 0, 0),  # p4    (0, 0, 0),  # p5    (0, 0, 1),  # p6]# Tensores de inércia (identidade mascarada -> símbolos na diagonal, zeros fora)inertia_template = [    [[1, 0, 0], [0, 1, 0], [0, 0, 1]] for _ in axis_order]auto = auto_parameters_from_order(    axis_order,    active_mask=active_mask,    excentricity_template=excentricity_template,    inertia_template=inertia_template,)q, qd = auto["q"], auto["qd"]print("q (gerados):", q)print("qd (gerados):", qd)

## 3. Calcular M, C, H e GUse as tabelas geradas acima diretamente em `equations_of_motion_from_order`. Bastasubstituir `axis_order` e os templates para outros robôs; todas as variáveis serãocriadas automaticamente.

In [ ]:
from robot_dynamics import equations_of_motion_from_order
from sympy import simplify

resultado = equations_of_motion_from_order(
    dh_params=auto["dh_params"],
    axis_order=axis_order,
    masses=auto["masses"],
    excentricities=auto["excentricities"],
    inertia_tensors=auto["inertia_tensors"],
    gravity=auto["gravity"],
    active_mask=active_mask,
    q=auto["q"],
    qd=auto["qd"],
)

M = simplify(resultado["M"])
C = simplify(resultado["C"])
H = simplify(resultado["H"])
G = simplify(resultado["G"])

# Retorne ou visualize cada componente individualmente (M, C, H, G)
M, C, H, G
